In [92]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import re

In [93]:
df = pd.read_csv('2022_LoL_esports_match_data_from_OraclesElixir.csv')

/tmp/ipykernel_52221/142356855.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2022_LoL_esports_match_data_from_OraclesElixir.csv')


In [94]:
print(df.columns.tolist())

['gameid', 'datacompleteness', 'url', 'league', 'year', 'split', 'playoffs', 'date', 'game', 'patch', 'participantid', 'side', 'position', 'playername', 'playerid', 'teamname', 'teamid', 'champion', 'ban1', 'ban2', 'ban3', 'ban4', 'ban5', 'pick1', 'pick2', 'pick3', 'pick4', 'pick5', 'gamelength', 'result', 'kills', 'deaths', 'assists', 'teamkills', 'teamdeaths', 'doublekills', 'triplekills', 'quadrakills', 'pentakills', 'firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim', 'team kpm', 'ckpm', 'firstdragon', 'dragons', 'opp_dragons', 'elementaldrakes', 'opp_elementaldrakes', 'infernals', 'mountains', 'clouds', 'oceans', 'chemtechs', 'hextechs', 'dragons (type unknown)', 'elders', 'opp_elders', 'firstherald', 'heralds', 'opp_heralds', 'void_grubs', 'opp_void_grubs', 'firstbaron', 'barons', 'opp_barons', 'firsttower', 'towers', 'opp_towers', 'firstmidtower', 'firsttothreetowers', 'turretplates', 'opp_turretplates', 'inhibitors', 'opp_inhibitors', 'damagetochampions', 'dp

In [95]:
numeric_df = df.select_dtypes(include=['float64', 'int64'])

In [97]:
df_cleaned = numeric_df.fillna(numeric_df.mean())

df_encoded = pd.get_dummies(df_cleaned, drop_first=True)
df_encoded[df_encoded.select_dtypes(include='bool').columns] = df_encoded.select_dtypes(include='bool').astype(int)

X = df_encoded.drop('result', axis=1)
y = df_encoded['result']
col = df.columns.sort_values()
firsts = []
for c in col:
    if re.findall(r'^first', c):
        firsts.append(c)
    if re.findall(r'(10)|(15)', c):
        firsts.append(c)
firsts
X = X[firsts]

In [98]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression()
model.fit(X_scaled, y)

feature_importance = (
  pd.Series(model.coef_[0], index=X.columns)
  .sort_values(ascending=False, key=abs)
)
most_important = feature_importance[:5].index.to_list()
print(most_important)
print(feature_importance.sort_values(key=abs, ascending=False)[:5])

['golddiffat15', 'firstbaron', 'assistsat15', 'opp_assistsat15', 'csdiffat15']
golddiffat15       0.941631
firstbaron         0.610066
assistsat15        0.365726
opp_assistsat15   -0.364530
csdiffat15        -0.195512
dtype: float64


In [99]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y
)

In [105]:
# Random Forest
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
rf.fit(X_train, y_train)
dt.fit(X_train, y_train)
print('Random Forest', knn.score(X_test, y_test))
print('Decision Tree', dt.score(X_test, y_test))
print('K Neighbors', knn.score(X_test, y_test))

Random Forest 0.601986878104497
Decision Tree 0.5953462427285042
K Neighbors 0.601986878104497


In [107]:
from sklearn.inspection import permutation_importance

# Random Forest feature importance
rf_importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Random Forest Most Important Features:")
print(rf_importances.head(5))

# Decision Tree feature importance
dt_importances = pd.Series(dt.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nDecision Tree Most Important Features:")
print(dt_importances.head(5))

# KNN feature importance (using permutation importance as an example)

knn_importances = permutation_importance(knn, X_test, y_test, n_repeats=10, random_state=42)
knn_importances_df = pd.Series(knn_importances.importances_mean, index=X.columns).sort_values(ascending=False)
print("\nKNN Most Important Features:")
print(knn_importances_df.head(5))

Random Forest Most Important Features:
golddiffat15    0.090028
xpdiffat15      0.065239
golddiffat10    0.055435
opp_goldat15    0.048431
goldat15        0.046799
dtype: float64

Decision Tree Most Important Features:
golddiffat15    0.185901
xpdiffat15      0.050725
golddiffat10    0.047213
xpdiffat10      0.046477
goldat10        0.043152
dtype: float64


KeyboardInterrupt: 

In [101]:
X_train_min = X_train[most_important]
X_test_min = X_test[most_important]

In [102]:
# rf_min = RandomForestClassifier()
rf_min = DecisionTreeClassifier(max_depth=3)
rf_min.fit(X_train_min, y_train)
print(rf_min.score(X_test_min, y_test))

0.6364650569766516
